In [15]:
!pip install chromadb==0.3.29
!pip install "pyautogen[lmm]~=0.2.0b4"
!pip install -U pydantic

  Using cached pydantic-1.10.13-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (149 kB)
Using cached pydantic-1.10.13-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.5.2
    Uninstalling pydantic-2.5.2:
      Successfully uninstalled pydantic-2.5.2
  Using cached pydantic-2.5.2-py3-none-any.whl.metadata (65 kB)
Using cached pydantic-2.5.2-py3-none-any.whl (381 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.3.29 requires pydantic<2.0,>=1.9, but you have pydantic 2.5.2 which is incompatible.
fastapi 0.85.1 requires pydantic!=1.7,!=1.7.1,!=1.7.2,!=1.7.3,!=1.8,!=1.8.1,<2

In [16]:
import os 
os.chdir('../')
import autogen
from prompts.plan_proposal import proposal_prompt
from functions.draft_proposal import store_draft_proposal_schema, store_draft_proposal

config_list_gpt4 = autogen.config_list_from_json(
    "/root/projects/proposal-gpt/OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)

# Phrase 1

In [17]:

common_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
}

draft_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
    "functions": [
        store_draft_proposal_schema
    ]
}

config_list_dalle = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["dalle"],
    },
)

user_proxy = autogen.UserProxyAgent(
   name="Admin",
   system_message="""A human admin. Interact with the bd_draft and bd_critic to draft the proposal, bd_critic will verify and give comments to improve the draft proposal content and structure. 
   The draft of proposal needs to be approved by this admin.
   """,
   code_execution_config=False,
)

draft = autogen.AssistantAgent(
    name="bd_draft",
    system_message=proposal_prompt,
    llm_config=draft_config,
)

draft.register_function(
    function_map={
        "store_draft_proposal": store_draft_proposal,
    }
)

critic = autogen.AssistantAgent(
    name="bd_critic",
    system_message="Critic. Double check the proposal from the executer and provide feedback.",
    llm_config=common_config,
)

groupchat = autogen.GroupChat(agents=[user_proxy, draft, critic], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=common_config)

input = """
<client_domain> (Client Domain): Healthcare

<client_problems_list> (Client Problems List):

Inefficient patient data management and retrieval.
High administrative workload for healthcare professionals.
Difficulty in early disease detection and diagnosis.
Limited access to real-time patient insights.
Privacy and security concerns related to patient data.
<solution> (Solution):

Proposed Solution Structure:
Component 1: Implement an AI-driven Electronic Health Record (EHR) system for efficient data management and retrieval.
Component 2: Develop a virtual assistant to automate administrative tasks and streamline workflows for healthcare professionals.
Component 3: Create a predictive analytics module for early disease detection and diagnosis.
Component 4: Build a secure patient portal for real-time access to medical records and insights.

<case_studies_list> (Case Studies List):

Case Study 1: Enhanced Patient Care

Description: Demonstrates how our AI-powered EHR system improved patient care and reduced errors in a hospital, resulting in a 20% reduction in readmissions.
Metrics: Reduced readmissions, improved patient outcomes, healthcare provider testimonial.
Case Study 2: Administrative Efficiency

Description: Highlights how our virtual assistant reduced administrative workload for a medical practice, leading to a 30% increase in appointment scheduling efficiency.
Metrics: Administrative time savings, appointment scheduling accuracy, client testimonial.
Case Study 3: Disease Detection

Description: Illustrates our success in early disease detection using predictive analytics, resulting in a 40% increase in early-stage diagnoses for a healthcare institution.
Metrics: Early-stage diagnoses, patient outcomes, healthcare provider feedback.

<success_stories_list> (Success Stories List):

Success Story 1: Revenue Growth

Description: Our AI-powered solutions contributed to a 15% increase in revenue for a healthcare client by improving patient care and operational efficiency.
Metrics: Revenue growth, ROI, client testimonial.
Success Story 2: Streamlined Operations

Description: We helped a medical practice achieve operational efficiency, reducing administrative costs by 25% and enhancing patient satisfaction.
Metrics: Cost savings, patient satisfaction scores, operational metrics.
Success Story 3: Data Security

Description: Our advanced security measures ensured compliance and data security for a healthcare organization, mitigating risks and enhancing trust.
Metrics: Data security compliance, risk mitigation, client feedback.
"""

user_proxy.initiate_chat(
    manager,
    message=input,
)

Admin (to chat_manager):


<client_domain> (Client Domain): Healthcare

<client_problems_list> (Client Problems List):

Inefficient patient data management and retrieval.
High administrative workload for healthcare professionals.
Difficulty in early disease detection and diagnosis.
Limited access to real-time patient insights.
Privacy and security concerns related to patient data.
<solution> (Solution):

Proposed Solution Structure:
Component 1: Implement an AI-driven Electronic Health Record (EHR) system for efficient data management and retrieval.
Component 2: Develop a virtual assistant to automate administrative tasks and streamline workflows for healthcare professionals.
Component 3: Create a predictive analytics module for early disease detection and diagnosis.
Component 4: Build a secure patient portal for real-time access to medical records and insights.

<case_studies_list> (Case Studies List):

Case Study 1: Enhanced Patient Care

Description: Demonstrates how our AI-powered 

ValueError: mode is only supported in Pydantic v2

In [ ]:
draft.chat_messages

# Phrase 2

In [ ]:
import autogen
from test_dalle import DALLEAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)

common_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
}

config_list_dalle = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["dalle"],
    },
)

dalle_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_dalle,
    "timeout": 120,
    "functions": [
        {
            "name": "store_image_url",
            "parameters": {
                "type": "object",
                "properties": {
                    "slide_title": {
                        "type": "string",
                        "description": (
                            "Enter the section title of the image."
                        ),
                    },
                    "url": {
                        "type": "string",
                        "description": (
                            "Enter the url of the image."
                        ),
                    }
                },
                "required": [
                    "slide_title",
                    "url"
                ]
            },
            "description": "This is a function allowing image_generator to input the section title and the image url.",
        }
    ]
}


def termination_msg(x): return isinstance(
    x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()


user_proxy = RetrieveUserProxyAgent(
    name="user",
    is_termination_msg=termination_msg,
    system_message="Assistant who has the content of draft proposal.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "code",
        "docs_path": "./draft_proposal.txt",
        "chunk_token_size": 1000,
        "model": config_list_gpt4[0]["model"],
        "client": chromadb.PersistentClient(path="./chromadb"),
        "collection_name": "draft_proposal",
        "get_or_create": True,
    },
    code_execution_config=False,  # we don't want to execute code in this case.
)

image_reviewer = autogen.AssistantAgent(
    name="image_reviewer",
    system_message="Image Reviewer. Double check the draft proposal from the executer and provide ideas of images for each sections. Try to provide clear description. Only response json array with each element has 2 key slide_title and image_description",
    llm_config=common_config,
)

image_generator = DALLEAgent(
    name="image_generator",
    system_message="Image Generator. Read the list slide title and image description and generate image for each slide.",
    llm_config=dalle_config,
)

def store_image_url(slide_title, url):
    print('store_image_url')
    print((slide_title, url))

image_generator.register_function(
    function_map={
        "store_image_url": store_image_url,
    }
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, image_reviewer, image_generator], messages=[], max_round=50)
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=common_config)

input_str = """Generate images for the proposal."""
user_proxy.initiate_chat(manager, problem=input_str)

ImportError: cannot import name 'OpenAIWrapper' from 'autogen' (/root/projects/proposal-gpt/.venv/lib/python3.10/site-packages/autogen/__init__.py)

# Phrase 3

In [ ]:
import autogen
from prompts.plan_proposal import proposal_prompt
from test_dalle import DALLEAgent
from functions.draft_proposal import store_draft_proposal_schema, store_draft_proposal
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb
import autogen
from prompts.plan_proposal import proposal_prompt
from test_dalle import DALLEAgent
from functions.draft_proposal import store_draft_proposal_schema, store_draft_proposal

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)
common_config = {
    "cache_seed": 43,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
}
def termination_msg(x): return isinstance(
    x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()
draft_config = {
    "cache_seed": 43,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
    "functions": [
        {
            "name": "store_final_proposal",
            "parameters": {
                "type": "object",
                "properties": {
                    "final": {
                        "type": "string",
                        "description": (
                            "Enter the final of proposal in markdown format."
                        ),
                    }
                },
                "required": [
                    "final"
                ]
            },
            "description": "This is a function allowing bd_final to input the final proposal to store in markdown format.",
        }
    ]
}

user_proxy = RetrieveUserProxyAgent(
    name="admin",
    is_termination_msg=termination_msg,
    system_message="Assistant who has the content of draft proposal and list image url.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    retrieve_config={
        "task": "code",
        "docs_path": ["./draft_proposal.txt", "./image.json"],
        "chunk_token_size": 4000,
        "model": config_list_gpt4[0]["model"],
        "client": chromadb.PersistentClient(path="./chromadb"),
        "collection_name": "final_proposal",
        "get_or_create": True,
    },
    code_execution_config=False,  # we don't want to execute code in this case.
)

draft = autogen.AssistantAgent(
    name="bd_final",
    system_message="""Act as a professional business development specialist in AI-powered software company.
Your goal is creating a compelling proposal slide deck for our client.
You will be provided draft proposal and list image and slide. 
Please ensure that each of these elements is presented in the proposal slides. Your goal is to create a persuasive and informative proposal that resonates with the client's needs and convinces them of the value we can provide.
Finally, store the draft of proposal in text format.""",
    llm_config=draft_config,
)

def store_final_proposal(draft):
    """
    Store draft of proposal in a local file.
    """
    # Exception
    if not isinstance(draft, str):
        raise TypeError("Draft of proposal must be a string.")
    # Function implementation...
    with open("./final_proposal.md", "w") as f:
        f.write(draft)
    # False case
    if not os.path.exists("./final_proposal.md"):
        return False
    return True


draft.register_function(
    function_map={
        "store_final_proposal": store_final_proposal,
    }
)

input = """Create final proposal with the provided content."""

user_proxy.initiate_chat(
    draft,
    problem=input,
)

Trying to create collection.


/Users/sibae/miniforge3/envs/autogen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2


doc_ids:  [['doc_1', 'doc_0']]
Adding doc_id doc_1 to context.
Adding doc_id doc_0 to context.
admin (to bd_final):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: Create final proposal with the provided content.

Context is: [
    {
        "slide_title": "Executive Summary",
        "url": "./images/Executive Summary.jpg"
    },
    {
        "slide_title": "Understanding Client Pain Points",
        "url": "./images/Understanding Client Pain Points.jpg"
    },
    {
        "slide_title": "Solution Overview",
        "url

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


bd_final (to admin):

***** Suggested function Call: store_final_proposal *****
Arguments: 
{
"final": "# Executive Summary\n\nOur proposal offers a unique, AI-powered solution that stands out in the market due to its ability to address your key challenges effectively. We aim to transform your healthcare organization by improving patient data management, reducing administrative workload, enabling early disease detection, providing real-time patient insights, and ensuring data privacy.\n\n![Executive Summary](./images/Executive Summary.jpg)\n\n# Understanding Client Pain Points\n\n- Inefficient patient data management and retrieval\n- High administrative workload for healthcare professionals\n- Difficulty in early disease detection and diagnosis\n- Limited access to real-time patient insights\n- Privacy and security concerns related to patient data\n\n![Understanding Client Pain Points](./images/Understanding Client Pain Points.jpg)\n\n# Solution Overview\n\nOur solution comprises four 

TypeError: 'NoneType' object is not subscriptable

In [ ]:
draft._reply_func_list

[{'trigger': [autogen.agentchat.agent.Agent, None],
  'reply_func': <function autogen.agentchat.conversable_agent.ConversableAgent.a_check_termination_and_human_reply(self, messages: Optional[List[Dict]] = None, sender: Optional[autogen.agentchat.agent.Agent] = None, config: Optional[Any] = None) -> Tuple[bool, Optional[str]]>,
  'config': None,
  'init_config': None,
  'reset_config': None},
 {'trigger': [autogen.agentchat.agent.Agent, None],
  'reply_func': <function autogen.agentchat.conversable_agent.ConversableAgent.check_termination_and_human_reply(self, messages: Optional[List[Dict]] = None, sender: Optional[autogen.agentchat.agent.Agent] = None, config: Optional[Any] = None) -> Tuple[bool, Optional[str]]>,
  'config': None,
  'init_config': None,
  'reset_config': None},
 {'trigger': [autogen.agentchat.agent.Agent, None],
  'reply_func': <function autogen.agentchat.conversable_agent.ConversableAgent.a_generate_function_call_reply(self, messages: Optional[List[Dict]] = None, sen

In [ ]:
user_proxy.generate_init_message(problem=input)

Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2


doc_ids:  [['doc_1', 'doc_0']]
Adding doc_id doc_1 to context.
Adding doc_id doc_0 to context.


'You\'re a retrieve augmented coding assistant. You answer user\'s questions based on your own knowledge and the\ncontext provided by the user.\nIf you can\'t answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.\nFor code generation, you must obey the following rules:\nRule 1. You MUST NOT install any packages because all the packages needed are already installed.\nRule 2. You must follow the formats below to write your code:\n```language\n# your code\n```\n\nUser\'s question is: Create final proposal with the provided content.\n\nContext is: [\n    {\n        "slide_title": "Executive Summary",\n        "url": "./images/Executive Summary.jpg"\n    },\n    {\n        "slide_title": "Understanding Client Pain Points",\n        "url": "./images/Understanding Client Pain Points.jpg"\n    },\n    {\n        "slide_title": "Solution Overview",\n        "url": "./images/Solution Overview.jpg"\n    },\n    {\n        "slide_title": "How the Solut